In [ ]:
import numpy as np
import argparse
import os
import imp
import re
import pickle
import datetime
import random
import math
import copy

from sklearn.model_selection import KFold, StratifiedKFold
import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset, Subset

from utils import metrics


### Base GRU model

In [ ]:
input_dim = 25
pad_token = np.zeros(input_dim)
def pad_sents(sents, pad_token):

    sents_padded = []

    max_length = max([len(_) for _ in sents])
    for i in sents:
        padded = list(i) + [pad_token]*(max_length-len(i))
        sents_padded.append(np.array(padded))

    return np.array(sents_padded)

In [ ]:
def get_bce_loss(y_pred, y_true):
    loss = torch.nn.BCELoss()
    return loss(y_pred[:,-1], y_true)

In [ ]:
def predict_all_visits_bce_loss(y_pred, x_lab_length, y_true):
    # print(y_pred.shape, y_true.shape)
    batch_size = len(y_true)
    loss = torch.nn.BCELoss()
    indices = torch.arange(batch_size, dtype=torch.int64)
    # print(type(indices), indices)
    losses = 0
    for i in indices:
        visits_indices = torch.arange(x_lab_length[i], dtype=torch.int64)
        patient_losses = 0
        for v in visits_indices:
            patient_losses += loss(y_pred[i][v], y_true[i])
            # try:
            #     patient_losses += loss(y_pred[i][v], y_true[i])
            # except:
            #     print('error loss:', (y_pred[i][v], y_true[i]))
        losses += patient_losses/x_lab_length[i]
    return losses/batch_size

In [ ]:
def predict_last_visit_bce_loss(y_pred, x_lab_length, y_true):
    batch_size = len(y_true)
    loss = torch.nn.BCELoss()
    indices = torch.arange(batch_size, dtype=torch.int64)
    losses = 0
    for i in indices:
        losses += (loss(y_pred[i][x_lab_length[i].long()-1], y_true[i]))
    return losses/batch_size

In [ ]:
class Dataset(data.Dataset):
    def __init__(self, x_lab, x_lab_length, x_demo, y):
        self.x_lab = x_lab
        self.x_lab_length = x_lab_length
        self.x_demo = x_demo
        self.y = y

    def __getitem__(self, index): # 返回的是tensor
        return self.x_lab[index], self.x_lab_length[index], self.x_demo[index], self.y[index]

    def __len__(self):
        return len(self.y)

In [ ]:
data_path = './dataset/tongji/processed_data/'
file_name = './ckpt/gru.pth'

batch_size = 64
num_epochs = 100
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
#device = torch.device('cpu')
print("available device: {}".format(device))


x_lab = pickle.load(open('./dataset/tongji/processed_data/train_x_labtest.pkl', 'rb'))
x_lab = np.array(x_lab, dtype=object)
x_lab = [torch.Tensor(_) for _ in x_lab]

x_demo = pickle.load(open('./dataset/tongji/processed_data/train_x_demographic.pkl', 'rb'))
x_demo = np.array(x_demo)

y = pickle.load(open('./dataset/tongji/processed_data/train_y_outcome.pkl', 'rb'))
y = np.array(y)

# print(len(x_lab[0][0]))
# x_lab = pad_sents(x_lab, pad_token)
# print(x_lab.shape)
x_lab_length = [len(_) for _ in x_lab]
x_lab_length = np.array(x_lab_length)
max_visits = x_lab_length.max()
x_lab = torch.nn.utils.rnn.pad_sequence((x_lab), batch_first=True)
# print(x_lab)
# print(x_lab.shape)
# x_lab = torch.nn.utils.rnn.pack_sequence((x_lab), enforce_sorted=False)
print(max_visits)
# len(x_lab[7])

In [ ]:
train_dataset = Dataset(x_lab, x_lab_length, x_demo, y)
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# print(x_lab.shape)
# print(x_demo.shape)
# print(y.shape)
# print(y)

# print(train_dataset.x_lab)


In [ ]:
class GRU(nn.Module):
    def __init__(self, input_lab_dim, input_demo_dim, hidden_dim, output_dim, act_layer=nn.GELU, drop=0.):
        super(GRU, self).__init__()

        # hyperparameters
        self.input_lab_dim = input_lab_dim
        self.input_demo_dim = input_demo_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.demo_proj = nn.Linear(input_demo_dim, hidden_dim)
        self.lab_proj = nn.Linear(input_lab_dim, hidden_dim)

        self.gru = nn.GRU(input_size = hidden_dim, hidden_size = hidden_dim, num_layers = 1, batch_first = True)
        
        self.act = act_layer()
        self.fc = nn.Linear(2*hidden_dim, output_dim)
        self.drop = nn.Dropout(drop)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x_lab, x_lab_length, x_demo):
        batch_size, max_length, input_dim = x_lab.shape

        x_lab = self.lab_proj(x_lab)
        x_lab = self.act(x_lab)
        # print('1', x_lab.shape)
        # x_lab = torch.nn.utils.rnn.pack_padded_sequence(x_lab, x_lab_length, batch_first=True, enforce_sorted=False)
        x_lab, h_n = self.gru(x_lab) # output: (batch_size,L,hidden_dim) h_n: (1, batch_size, hidden_dim)
        x_lab = x_lab # (batch_size, hidden_dim)
        # print('2', x_lab.shape)

        x_demo = self.demo_proj(x_demo)
        x_demo = self.act(x_demo) # (batch_size, hidden_dim)

        x_demo = torch.reshape(x_demo.repeat(1, max_length), (batch_size, max_length, self.hidden_dim))

        # print(x_lab.shape, x_demo.shape)
        x = torch.cat((x_lab, x_demo), 2) # (batch_size, 2*hidden_dim)

        # print(x.shape)
        x = self.drop(x)
        x = self.fc(x)
        x = self.drop(x)
        
        x = self.sigmoid(x)
        return x


In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED) # numpy
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED) # cpu
torch.cuda.manual_seed(RANDOM_SEED) # gpu
torch.backends.cudnn.deterministic=True # cudnn
np.set_printoptions(threshold=np.inf, precision=2, suppress=True)


def train_epoch(model, device, dataloader, loss_fn, optimizer):
    train_loss = []
    model.train()
    for step, data in enumerate(dataloader):   
        batch_x_lab, batch_x_lab_length, batch_x_demo, batch_y = data
        batch_x_lab, batch_x_lab_length, batch_x_demo, batch_y = batch_x_lab.float(),batch_x_lab_length.float() , batch_x_demo.float().to(device), batch_y.float()
        batch_y = batch_y.unsqueeze(-1)
        optimizer.zero_grad()
        output = model(batch_x_lab, batch_x_lab_length, batch_x_demo)
        # print('train epoch output', output)
        loss = loss_fn(output, batch_x_lab_length, batch_y)
        # print('train epoch output', output, batch_x_lab_length, batch_y)
        train_loss.append(loss.item())
        loss.backward()
        optimizer.step()
    return np.array(train_loss).mean()

def val_epoch(model, device, dataloader, loss_fn):
    val_loss = []
    y_pred = []
    y_true = []
    evaluation_scores = {}
    model.eval()
    with torch.no_grad():
        for step, data in enumerate(dataloader):   
            batch_x_lab, batch_x_lab_length, batch_x_demo, batch_y = data
            batch_x_lab, batch_x_lab_length, batch_x_demo, batch_y = batch_x_lab.float(),batch_x_lab_length.float() , batch_x_demo.float().to(device), batch_y.float()
            batch_y = batch_y.unsqueeze(-1)
            output = model(batch_x_lab, batch_x_lab_length, batch_x_demo)
            loss = loss_fn(output, batch_x_lab_length, batch_y)
            val_loss.append(loss.item())

            # output = torch.squeeze(output)

            # y_pred += [output[i][:batch_x_lab_length[i].long()].mean().item() for i in range(len(batch_y))]
            for i in range(len(batch_y)):
                # print('to extend!---', output[i][:batch_x_lab_length[i].long()])
                y_pred.extend(output[i][:batch_x_lab_length[i].long()].tolist())
                # print('-batchy---', [batch_y[i].item()] * batch_x_lab_length[i].long())
                y_true.extend([batch_y[i].item()] * batch_x_lab_length[i].long())
    # print('!!!!!!!!!a!!!23', y_true, y_pred)
    y_true = np.array(y_true)
            # y_true += list(batch_y.cpu().numpy().flatten())
    
    y_pred = np.array(y_pred)
    # print('shape here:!---', y_true.shape, y_pred.shape, y_true, y_pred)
    # print('a---', y_pred.shape, y_pred)
    y_pred = np.stack([1 - y_pred, y_pred], axis=1)
    # print('b---', y_pred.shape, y_pred)
    evaluation_scores = metrics.print_metrics_binary(y_true, y_pred)
    return np.array(val_loss).mean(), evaluation_scores

def test_epoch(model, device, dataloader, loss_fn):
    test_loss = []
    y_pred = []
    y_true = []
    evaluation_scores = {}
    model.eval()
    with torch.no_grad():
        for step, data in enumerate(dataloader):   
            batch_x_lab, batch_x_lab_length, batch_x_demo, batch_y = data
            batch_x_lab, batch_x_lab_length, batch_x_demo, batch_y = batch_x_lab.float(),batch_x_lab_length.float() , batch_x_demo.float().to(device), batch_y.float()
            batch_y = batch_y.unsqueeze(-1)
            output = model(batch_x_lab, batch_x_lab_length, batch_x_demo)
            loss = loss_fn(output, batch_x_lab_length, batch_y)
            test_loss.append(loss.item())
            
            for i in range(len(batch_y)):
                # print('to extend!---', output[i][:batch_x_lab_length[i].long()])
                y_pred.extend(output[i][:batch_x_lab_length[i].long()].tolist())
                # print('-batchy---', [batch_y[i].item()] * batch_x_lab_length[i].long())
                y_true.extend([batch_y[i].item()] * batch_x_lab_length[i].long())
    y_true = np.array(y_true)
            # y_true += list(batch_y.cpu().numpy().flatten())
    y_pred = np.array(y_pred)
    # print('shape here:!---', np.array(y_true).shape, y_pred.shape, y_true, y_pred)
    y_pred = np.stack([1 - y_pred, y_pred], axis=1)
    evaluation_scores = metrics.print_metrics_binary(y_true, y_pred)
    return np.array(test_loss).mean(), evaluation_scores

# train_epoch(model, device, train_loader, get_bce_loss, optimizer)

In [ ]:
model = GRU(input_lab_dim=25, input_demo_dim=2, hidden_dim=32, output_dim=1, act_layer=nn.GELU, drop=0.).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = predict_all_visits_bce_loss
train_loader = DataLoader(train_dataset, batch_size=128)
train_loss = train_epoch(model, device, train_loader, criterion, optimizer)
# val_loss = test_epoch(model, device, train_loader, criterion)

### Train, Validate and Test

In [ ]:
all_history={}
test_performance={'test_loss': [],'test_accuracy':[], 'test_auroc':[], 'test_auprc':[]}
dataset = train_dataset
num_folds = 10
kfold_test = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=RANDOM_SEED)
for fold_test, (train_and_val_idx, test_idx) in enumerate(kfold_test.split(np.arange(len(dataset)), dataset.y)):
    print('====== Test Fold {} ======'.format(fold_test + 1))
    kfold_val = StratifiedKFold(n_splits=num_folds-1, shuffle=True, random_state=RANDOM_SEED)
    sub_dataset = Dataset(dataset.x_lab[train_and_val_idx], dataset.x_lab_length[train_and_val_idx], dataset.x_demo[train_and_val_idx], dataset.y[train_and_val_idx])
    all_history['test_fold_{}'.format(fold_test+1)] = {}
    for fold_val, (train_idx, val_idx) in enumerate(kfold_val.split(np.arange(len(sub_dataset)), sub_dataset.y)):
        print('Val Fold {}'.format(fold_val + 1))
        train_sampler = SubsetRandomSampler(train_idx)
        val_sampler = SubsetRandomSampler(val_idx)
        train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
        val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)
        model = GRU(input_lab_dim=25, input_demo_dim=2, hidden_dim=32, output_dim=1, act_layer=nn.GELU, drop=0.).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        criterion = predict_all_visits_bce_loss # get_bce_loss
        history = {'train_loss': [], 'val_loss': [],'val_accuracy':[], 'val_auroc':[], 'val_auprc':[]}
        # history = {}
        for epoch in range(num_epochs):
            train_loss=train_epoch(model, device, train_loader, criterion, optimizer)
            val_loss, val_evaluation_scores=val_epoch(model, device, val_loader, criterion)
            # save performance history on validation set
            print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Val Loss:{:.3f}".format(epoch + 1, num_epochs, train_loss, val_loss))
            history['train_loss'].append(train_loss)
            history['val_loss'].append(val_loss)
            history['val_accuracy'].append(val_evaluation_scores['acc'])
            history['val_auroc'].append(val_evaluation_scores['auroc'])
            history['val_auprc'].append(val_evaluation_scores['auprc'])
        all_history['test_fold_{}'.format(fold_test+1)]['fold{}'.format(fold_val+1)] = history
    
    # at the same epoch, calculate average auroc of each fold, set the best one as the best model
    best_epoch = 0
    best_performance = 0.0
    for step in range(num_epochs):
        auroc_list = []
        for fold in range(num_folds-1):
            auroc_list.append(all_history['test_fold_{}'.format(fold_test+1)]['fold{}'.format(fold+1)]['val_auroc'][step])
        performance = np.array(auroc_list).mean()
        if performance > best_performance:
            best_performance = performance
            best_epoch = step + 1
        print('Epoch:{}/{} AVG AUROC:{:.3f}'.format(step+1, num_epochs, performance))

    print(f"Best epoch: {best_epoch} Best performance: {best_performance}")
    print('Begin to evaluate the best model on test set')
    # train model again with train_and_val set (num_epochs = best_epoch), then test on test dataset 
    train_sampler = SubsetRandomSampler(train_and_val_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
    model = GRU(input_lab_dim=25, input_demo_dim=2, hidden_dim=32, output_dim=1, act_layer=nn.GELU, drop=0.).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = predict_all_visits_bce_loss # get_bce_loss
    for epoch in range(best_epoch):
        train_loss=train_epoch(model, device, train_loader, criterion, optimizer)
        print("Epoch:{}/{} AVG Training Loss:{:.3f}".format(epoch + 1, num_epochs, train_loss))
    test_loss, test_evaluation_scores = test_epoch(model, device, test_loader, criterion)
    test_performance['test_loss'].append(test_loss)
    test_performance['test_accuracy'].append(test_evaluation_scores['acc'])
    test_performance['test_auroc'].append(test_evaluation_scores['auroc'])
    test_performance['test_auprc'].append(test_evaluation_scores['auprc'])
    print(f"Performance on test set {fold_test+1}: Accuracy = {test_evaluation_scores['acc']}, AUROC = {test_evaluation_scores['auroc']}, AUPRC = {test_evaluation_scores['auprc']}")
    


In [ ]:
# Calculate average performance on 10-fold test set
print(test_performance)
test_accuracy_list = np.array(test_performance['test_accuracy'])
test_auroc_list = np.array(test_performance['test_auroc'])
test_auprc_list = np.array(test_performance['test_auprc'])
print('accuracy: mean={:.3f}, std={:.3f}'.format(test_accuracy_list.mean(), test_accuracy_list.std()))
print('auroc: mean={:.3f}, std={:.3f}'.format(test_auroc_list.mean(), test_auroc_list.std()))
print('auprc: mean={:.3f}, std={:.3f}'.format(test_auprc_list.mean(), test_auprc_list.std()))

"""
accuracy: mean=0.956, std=0.025
auroc: mean=0.986, std=0.015
auprc: mean=0.984, std=0.018
"""

In [ ]:
criterion = predict_last_visit_bce_loss

all_history={}
test_performance={'test_loss': [],'test_accuracy':[], 'test_auroc':[], 'test_auprc':[]}
dataset = train_dataset
num_folds = 10
kfold_test = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=RANDOM_SEED)
for fold_test, (train_and_val_idx, test_idx) in enumerate(kfold_test.split(np.arange(len(dataset)), dataset.y)):
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
    test_sampler = SubsetRandomSampler(test_idx)
    test_loss, test_evaluation_scores = test_epoch(model, device, test_loader, criterion)
    test_performance['test_loss'].append(test_loss)
    test_performance['test_accuracy'].append(test_evaluation_scores['acc'])
    test_performance['test_auroc'].append(test_evaluation_scores['auroc'])
    test_performance['test_auprc'].append(test_evaluation_scores['auprc'])
    print(f"Performance on test set {fold_test+1}: Accuracy = {test_evaluation_scores['acc']}, AUROC = {test_evaluation_scores['auroc']}, AUPRC = {test_evaluation_scores['auprc']}")

# Calculate average performance on 10-fold test set
print(test_performance)
test_accuracy_list = np.array(test_performance['test_accuracy'])
test_auroc_list = np.array(test_performance['test_auroc'])
test_auprc_list = np.array(test_performance['test_auprc'])
print('accuracy: mean={:.3f}, std={:.3f}'.format(test_accuracy_list.mean(), test_accuracy_list.std()))
print('auroc: mean={:.3f}, std={:.3f}'.format(test_auroc_list.mean(), test_auroc_list.std()))
print('auprc: mean={:.3f}, std={:.3f}'.format(test_auprc_list.mean(), test_auprc_list.std()))

"""
accuracy: mean=0.986, std=0.022
auroc: mean=0.996, std=0.009
auprc: mean=0.996, std=0.009
"""
